# 2A.bonus
Before we get to the "real" assigment, you get a small optional bonus introduction to creating python functions. If you are used to creatng funcitons, you can skip or delete this section. 

When we create a function in python we encapsel a number of operations and objects inside a single piece of code. This is nice if we want to use that code more than once - perhaps simply with some parameters change. But at least as importantly is the fuctions ability to contain tempoary objects. Containing objects is a big deal. Everything defined inside functions, only exists there. If you try to print objects defind inside functions you will see that they are "not defined" (unless you have specifically asked the function to define them globally). This is because they only exist in there respective functions. Think of it as: "what happens in vegas stays in vegas". Only that which you specifically want to take home from vegas gets home. Everything else is forgotten. 

As such, functions are very good at creating a hygenic coding environment less prone to errors. They are less prone to error because we don't risk using some old objects (which we might arbitrarily have called x, i, j or test_thing) in new code unknowingly. Instead of the code runnnig and giving us wierd results or arcane errors, we will get a "not defiened error" or similar which is often much easier to debug. Functions are also more memory efficient since we don't store objects in memory for long. Especially if we do large calculations or have a lot of intermidiate dataframes (which we did in 1A and 1B) functions can help a lot and might save us from getting "memory errors" (a cheap solution compared to buying more ram or a new computer). 

You can read more about python functions here https://www.programiz.com/python-programming/function and a thousand other places. And as a minor note I like to use "camalCase" for function names and "under_score" for other objects. But there are a myriad of different conventions out there.

Below is an example of a function. It does not take any inputs but outs the sum of 2+2. Quite useless, but it serves to highlight four things; **the declaration, the documentation, a temporary objet and the return() call**.

In [ ]:
def someFunction1(): # the declaration. You can call your function whatever, but start with def then the name the a parantasis that might take parameters and a :
    
    """This funcitons returns the result of 2+2""" # the documantion. Write what the function does (if it is too not trival/obvious) with 3 " on each side
    
    output = 2+2 # crate som object which only exist in the function
    
    return(output) # the the function to return that object

In [ ]:
someFunction1() # we call the function and we se our object! We can of course save it to a new object if we want to.

The nest function does take parameters. Two numbers.

In [ ]:
def someFunction2(x1,x2):
    
    """This funcitons takes as input two numbers x1 and x2. It returns the result of x1+x2"""

    output = x1+x2
    
    return(output)

In [ ]:
someFunction2(3,2) # try changing the numbers

The next function takes a list as its first parameter and a number as its second.

In [ ]:
def someFunction3(list_of_numbers, scalar):
    
    """This funcitons takes as input a list of numbers (list_of_numbers) and one number (scalar) x1 and x2.
    It outputs the result of elementwise products: list_of_numbers_i * scalar """

    
    for i in list_of_numbers:
        
        new_number = i * scalar
        
        print(new_number)
        
    print('Done') # note that the print statements run without any return call. Plots and changes to exiting objects also don't need a return call

In [ ]:
# feel free to change the list or number
list_n = [0,1,2,3,4] 
someFunction3(list_n, 2) 

We can also supply defualt values to our functions:

In [ ]:
def someFunction4(list_of_numbers, scalar = 5):
    
    """This funcitons is the same as the one above, but here the scalar is set to 5 by defaults.
    So if we don't input a value for scalar, the function just assumes we want 5"""

    
    for i in list_of_numbers:
        
        new_number = i * scalar
        
        print(new_number)
        
    print('Done') # note that the printstatements runs without any return statement

In [ ]:
someFunction4(list_n) # the scalar is 5 by default, but you can but in a new scalar if you want to

Now, If you try to print objects such as "output" of "new_number" now you will see they are "not defined". This is because they only exist in their respective functions. Conversely "list_n" was created outside our functions so that does exist. Some code in the assigment below is already in functions and I encourage you to see if you can embed even more code into functions as well. 

You can also try making you own small function here - it does not hurt to do anything useful ;) 

# 2A.for real:

So, we now begin the actual assigment. The focus here will be:

- Introducing the concept out-of-sample prediction
- Introducing sliding window/lag models
- Creating the best_binary, best_binary_m1, best_binary_m2, best_binary_m3, best_binary_4, best_binary_m5 feature/tagets
- Partitioning the data in train, validation and test set
- Generate some baseline "models" (models here being used extremely generously). Specifically  a *persitance* and a *ratio* baseline model.
- Test our baseline models accuracy 

We will take it one step at a time, but start by importing all the libraries we need: numpy as np, pandas as pd, geopandas as gpd, matplotlib.pyplot as plt and pickle. From scipy you should also import stats


In [ ]:
import ...
import ...
import ...
import ...
import ...

from ... import ...

We also set our numpy seed, so "random" will be the same random each time

In [ ]:
 np.random.seed(seed=42)

# 2A.0

Hopefully you have read some of the articles on the curriculum and have already been exposed to the concept out-of-sample prediction. But to reiterate: When we focus on forecasting over hypothesis testing, estimating parameters and testing significance is rarely central. Furthermore, if the goal is purely heuristic, concerns about bias and endogeneity are shelved. Instead, the primary concern is increasing prediction-power. We do this by mitigating both *underfitting* and *overfitting*. Underfitting, meaning creating a model which has failed to learn any salient patterns in the data pertaining to the phenomenon under investigation. Overfitting, meaning the accidental identification of patterns present in the data, yet not present in the real world. Out-of-sample prediction is a way of evaluating our models which reveals overfitting.

A model that has overfitted to some data might be very good at "predicting" observaitions which it has already seen but very poor at predicting $y_{new}$ given some $X_{new}$. Setting aside a subset of data that will not be used to train our model allow us to use said subset as a *test_set*. this test_set can then be used to evaluate our model(s). This will reveal any overfitting since the test_set was not used in the training effort. You can read more about overfitting here:

- https://en.wikipedia.org/wiki/Overfitting
- https://towardsdatascience.com/what-are-overfitting-and-underfitting-in-machine-learning-a96b30864690

Often we will also need a validation set. This is simply a "test set" which we use along the way to test variouse decision regarding our models. Thus, the actual test set can be kept apart from any testing until the very end where it will be used to test our final model(s). The reason we use a validation set is that we might be overfitting somewhat as we tweak our model along the way: That is, we might overfit to the validation set via the choices we make along our feature engineering and model construction efforts. Saving the test set for last will expose such issues. Naturally, the more distinct hold-out sets you can use along the way, the less a risk of overfitting (we will talk about something called k-fold-cross-validation later). For now, however, we will simply partition our data into one `test_set`, one `val_set` and one `training_set`. The latter is what we will use to train our model(s).  

Often some random subset(s) is simply pulled from the data to act as test and validaiton set. In timeseries, however, that does not make sense. Here the validation and test set must be furture in time compared to the training set.


**How can we spot underfitting?**

The code below is not a very good representation of overfitting, but it does illustrate the concept of test and train sets and how out-of-sample prediction can lead to honest evaluations in forecasting. In the first exemple the testset is chosen at random from the full set. You don't have to change the function, but you are very welcome to.

In [ ]:
def linTestTrainEx():


    n_samples = 100

    x = np.linspace(0, 10, num=n_samples)
    ε = 8
    y = np.random.normal(4+5*x, ε) # a linear reletionship given by y = 4 + 5x + ε

    y[-20:-1] = y[-20:-1] * 1.4

    test_idx = np.random.randint(0, n_samples, int(n_samples/10))

    y_test = y[test_idx]
    y_train = np.delete(y,test_idx)

    x_test = x[test_idx]
    x_train = np.delete(x,test_idx)

    slope, intercept, r_value, p_value, std_err = stats.linregress(x_train, y_train)
    
    plt.plot(x, intercept + slope*x, 'r', label='fitted line')
    plt.plot(x_train,y_train, 'o', label = 'train data')
    plt.plot(x_test,y_test, 'o', label = 'test data')
    
    plt.xlabel('some time intervals')
    plt.ylabel('some measurement')
    
    plt.legend()
    plt.show()

In [ ]:
linTestTrainEx()

So, here we just took out a random subset for testing, and eyeballing the plot it seems to the test set (orange) as weel as the trainig set (blue). However, in a real forcasting senario, we would want to know how our model far against data from a new time interval - not just random points. Thus, in the next exemple we only train on the first 4 perioeds and use the last as test set. You don't have to change the function, but you are very welcome to.

In [ ]:
def quadTestTrainEx():

    n_samples = 100

    x = np.linspace(0, 10, num=n_samples)
    ε = 8
    y = np.random.normal(4+5*x, ε) # a linear reletionship given by y = 4 + 5x + ε

    y[-20:-1] = y[-20:-1] * 1.4
    
    test_idx = np.arange(n_samples - 25 , n_samples, 1)

    y_test = y[test_idx]
    y_train = np.delete(y,test_idx)

    x_test = x[test_idx]
    x_train = np.delete(x,test_idx)

    slope, intercept, r_value, p_value, std_err = stats.linregress(x_train, y_train)
    
    plt.plot(x, intercept + slope*x, 'r', label='fitted line')
    plt.plot(x_train,y_train, 'o', label = 'train data')
    plt.plot(x_test,y_test, 'o', label = 'test data')
    
    plt.xlabel('some time intervals')
    plt.ylabel('some measurement')
    
    plt.legend()
    plt.show()

In [ ]:
quadTestTrainEx()

Now it should be clear that our model fits the train set (blue) better than the test set (orange). This is a problem, but we would not know it if we had used a random subset as test set. There are other reasons we would not want to use a random subset, but this one is central.

**Can you give other reasons why we should not just use random subsets of data for the validation and (especially) test set when we are working with timeseries?**

In general, overfitting can happen when we start having more complex patterns and use models with greater capacity (ability to fit complex models). Out-of-sample gives us honest evaluations when we predict. It is especially important when we do forecasting since we know the furture is not a random subset of the present. But really, out-of-sample prediction is always important.

# 2A.1

There a many different ways to make predictions about the future in machine learning and statistics. One rather (at least conceptual) simple metod is the sliding window model (also called to rolling window model, the lag model or lead model - not precisly the same things, but pretty close.). The term "model" might here be a bit confusing since it is not one specific kind of machine learning model but more of an approach (if you ask me). The idea is simply that we shifts our features X relative to our target y. Thus instead of using our features X to predict y we try to predict y next year, or the year after that ($y_{t+n}$). I'm not going to go explain a lot here, but I can recommend the following (very short) blogs. You should at least skim the first one if you are not familiar with *lags* and *leads*. 

https://machinelearningmastery.com/time-series-forecasting-supervised-learning/   
https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/   
https://towardsdatascience.com/how-not-to-use-machine-learning-for-time-series-forecasting-avoiding-the-pitfalls-19f9d7adf424   
https://www.linkedin.com/pulse/how-use-machine-learning-time-series-forecasting-vegard-flovik-phd-1f/
https://medium.com/open-machine-learning-course/open-machine-learning-course-topic-9-time-series-analysis-in-python-a270cb05e0b3


Will start with a small exemple. Create a empty pandas datafram (`df = pd.DataFrame()`) and add two features in said dataframe using the code below:

In [ ]:
df = ...
df['x']  = np.arange(1989,1999,1)
df['y'] = np.arange(0,20,2)

df

So x is years and y some value we are interested in forecsting. Create 3 new targets by shifting `y` respectively -1, -2 and -3 three years, e.g: `df['y_t1'] = df['y'].shift(-1)`

In [ ]:
df['y_t1'] = df['y'].shift(...)
df[...] = df[...].shift(...)
df[...] = df[...].shift(...)

Look at your dataframe, i.e. just run `df`

Look closely at the dataframe. Given our three new targets we can now - in theory - use x and y to predict $y_{t1}$, $y_{t2}$ and $y_{t3}$. That is; we can (try to) predict the value of y 1, 2 and 3 years into the furture. But we have also introduced soom NaNs. There should be 1 NaN in $y_{t1}$, 2 NaNs in $y_{t2}$ and 3 NaNs in $y_{t3}$.

**Why do we get these NaNs?**

As you might then guess, the further out in to the future we want to train, the furture back into time you must step to train our model. Thus if I where standing in 2014 (and I have data from this year and some number of years before) I could use data from 2013 and back to create a model that predicts one year ahead. But to predict two years ahead I would have the go back to 2012. I need to do this to get corresponding targets. 


Here we can ignore this issue since the UCDP data on conflict deaths ('best': what we will devrive our target(s) from) continues beyound the years of the PRIO data (which contains all our structural features). UCDP has last year = 2019 and PRIO has last year = 2014). So even with our last obs in PRIO (from 2014) we have UCDP data "enough" to forecast 5 years into the future. This can be a bit of a mind bender, but try to think about it - even drawing it out for yourself might help. 

**Imagine if we wanted to predict 6 and not 5 years ahead? what would happpen? How could we do it?**


Now simply remove the rows with NaNs (this is not necessarily the way to go, but it is simple and we can it here because of the difference between the data from UCDP and PRIO). Here you can do this be making a new dataframe where you only keep the first 7 obs: `df_cut = df.iloc[:7]`

Notice the method .iloc() (index location). If you have read VanderPlas you should be familiar with it. If not it might be a bit arcane. You can (should) look in your VandarPlas book or read more about it here 
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html

In short iloc replicates what we do in numpy with some_1d_array[:7] where we take all elements up to the 7th. In contrast .loc\[:7\] (no "i" in front) would would take all elements up til the on called '7' (if such one exist). The difference here comes from the fact the numpy arrays only have index, so for a matrix it has i and j. But pandas dataframes have both actual index values (e.g i and j) but can also have named or spacially number index (e.g. columns can have feature names and the rows might not have a index name/number corrosponding the the actual index..). Sometimes this makes a big difference - sometimes not so much. It migh be a bit confusing, but once you get use to it, it does makes sense.

Look at `df_cut`

So the sliding window model (or appraoch) is what we will use going forward. We do this becaues it is comceptually simple and it allow us to use any supervised machine learning algorithme for our porpuse. That is we don't have to use annything timeseries specific. On a more technical note: Using this appraoch we do not have to worry to much about non-stationarity since we will create a seperate model for each year+n (if that does not make sense, don't worry about it). 

On the flip side it is not a very elegant appraoch: We have to train new models all the time, we are discarding a lot of data and we do not share information between models. There are models and appraoches that ammends or bypasses these cons such a Baysian mulity-level models, Gaussain Processes and some Atificial Naural Networks. We do not have time to go deep in to these models in this course, but I will set aside some time to speak about them at the end of the semester.

For now we are gonna ignore all issues with the sliding window appraoch for the sake of simplicity - but we will know that we ignore these issues and be refletive about it and don't forget it! These issues and limitations are important and will lead to a lot of confusion and error if you try to do conflcit prediction in a "real-life" setting without taking them into account.

# 2A.2

Before implimenting the lag approach on our actual data. we will quickly load our final_df.pkl (which we created in 1B) and create the target/feature `binary_best`.


To elaborate a bit on `binary_best`: In this course we are gonna focus on predicting the presence of conflict $\in \{0,1\}$. Not the magnitude. So our question is wheter there is conflict in some cell or not. This is called a classification task and give a binary target $y \in \{0,1\}$ classification entails predicting either a class $\hat{y}\in \{0,1\}$ or a probability $\hat{y}\in (0,1)$ of a class. The machine learning algorithmes we use later on will mostly produce for probabilities. I always prefer probabilities since there are more informative than hard binary predictions.

(Note: There are two simple reasons we don't try to predict conflcit magnitude (a regression task instead of a classsificatin task). 1) most of the litterature deals with classificaiton at the moment, so to have a "conversation" with that we also stick with classification. 2) time constrains.)  

So, since we do classification we need to convert our target `best` into a binary target $y \in \{0,1\}$ where 0 = no conflcit and 1 = some conflict.   

Start by loading your pickle `final_df.pkl`


Do some of your regular test, and maybe plot one or two year/feature(s) you feel familar with (could be 1994/log_best)

Now, lest see how to turn best binary. It is pretty simply. Look at the code below:

In [ ]:
np.unique((final_df['best'] > 0)*1)

`final_df['best'] > 0` gives a full array of best, but instead of the best values, we get booleans (True/False) regarding wither said score is above 0. The multiplication with 1 is a simple hack that turns the booleans into binaries (Flase  = 0 and True = 1). The np.unique() functions is not part of the binarization, but shows the result by returning all unique values which left in the array: As you see we only have 1's and 0's.

Lets use this trick `(final_df['best'] > 0)*1` to create a new feature in our final_df called `binary_best`

In [ ]:
final_df[...] = ...

Try to plot a year or two of binary_best to confirm the tis measure is indeed a binary indicator of conflict.

(Note: Naturally there is a lot more information in the best and log_best so we will use those (along low, high, and all the prio features) as predictors/features. but we'll get to that later.)

# 2A.4
Now we will create and trim our data (almost) just as we did in 2A.1. Specifically, we will create

- `best_binary_t1` by shifting binary_best -1
- `best_binary_t2` by shifting binary_best -2
- `best_binary_t3` by shifting binary_best -3
- `best_binary_t4` by shifting binary_best -4
- `best_binary_t5` by shifting binary_best -5

Note that the code look slightly different from the exemple in 2A.1. This is to take acount of each `gid` cell. Thats is we want to lag binary best for each cell alone. Not for all cells together. If that does not make sens take a minute to think about it. Taking account of cells is a reacurring thing when working with this kind of data (also when we interplolated you might remember).

In [ ]:
final_df['binary_best_t1'] = final_df.sort_values('year').groupby(['gid'])['binary_best'].shift(-1)
final_df[...] = final_df.sort_values(...).groupby([...])[...].shift(...)
final_df[...] = final_df.sort_values(...).groupby([...])[...].shift(...)
final_df[...] = final_df.sort_values(...).groupby([...])[...].shift(...)
final_df[...] = final_df.sort_values(...).groupby([...])[...].shift(...)


# Leading the binary target:
# df['lead_binary_best'] = df.sort_values('year').groupby(['gid'])['binary_best'].shift(-1)

Creat a couple of plots to see that you succeded. For instance `binary_best` in 1994 should look exactly like `binary_best_t1` in 1993 and `binary_best_t2` and so on. See if it checks out.

We can test wheter we succeded in more ways. Try runnig the code below. If everything is done right, you should see the disting "step-pattern" of NaNs for each random gid drawn:

In [ ]:
for gid in final_df['gid'].sample(6):
    
    test_tail = final_df[final_df['gid']==gid][['binary_best', 'binary_best_t1', 'binary_best_t2', 'binary_best_t3', 'binary_best_t4', 'binary_best_t5']].tail(6)
    print(f'gid: {gid}')
    print(test_tail.to_string())

Do you see the distinct pattern of NaNs? if not something might be amiss..

Another way to check it is with the code just below:

In [ ]:
list(final_df[final_df['year'] == 1993]['binary_best_t1']) == list(final_df[final_df['year'] == 1994]['binary_best'])

**Does the code just above make sense?**
**What does it do?**
**Do you get a True?**

Now lest trim our data just as we did in 2A.1. Here we discard everything after 2015. that is, we create a new datafram wich only have our obs from years (`year`) below 2015 (`<2015`).

In [ ]:
trim_df = final_df[final_df[...]< ...].copy() # copy to make it a new dataset and not just a slice of the old one - more robust

In [ ]:
trim_df.shape

We can run our loop agian to see that the distinct pattern of NaNs is gone:

In [ ]:
for gid in trim_df['gid'].sample(6):
    
    test_tail = trim_df[trim_df['gid']==gid][['binary_best', 'binary_best_t1', 'binary_best_t2', 'binary_best_t3', 'binary_best_t4', 'binary_best_t5']].tail(6)
    print(f'gid: {gid}')
    print(test_tail.to_string())

Now check if there are any NaNs left in our data set by using:   
`trim_df.isna().any()`

It seems like some of the dynamic features we did not interpolate does have some missings. Lest see how many:

In [ ]:
print(trim_df['ttime_mean'].isna().sum())
print(trim_df['gwno'].isna().sum())
print(trim_df['gwarea'].isna().sum())
print(trim_df['mountains_mean'].isna().sum())
print(trim_df['bdist2'].isna().sum())

Ok, so for three of our features it os only 15 obs. For two of our features it is a bit more. We can visualize our missings as follows (Missing are dark blue):

In [ ]:
# see more at https://geopandas.readthedocs.io/en/latest/docs/user_guide/mapping.html
missing_list = ('ttime_mean', 'gwno', 'gwarea', 'mountains_mean', 'bdist2')

for feature in missing_list:
    
    trim_df[trim_df['year']==1989].plot(column=feature, figsize=(15, 15), cmap='OrRd', missing_kwds = {'color':"blue"}) # the few missing in gwno, gwarea, bdist2 are in 89,91,92, and 93

- Ok, so the 15 missing in gwno, gwarea and bdist2 just seems to be some small islands in the first years. So we can just use `bfill` here (Remeber from 1B? else se pandas fillna documantation)
- the missings of ttime_mean are some costal points across all years. We here just set that the mean of the feature  
- the missing for mountains is also coastal points (is a lot more), but we can assume the are pretty close to the ocean. Thus we set these to the minimun value of the feature mountain

You don't have to change any code below, but feel free to argue why some other filling strategies might be more prudent.

In [ ]:
trim_df['ttime_mean'].fillna(trim_df['ttime_mean'].mean(), inplace=True)
trim_df['gwno'].fillna(method = 'bfill', inplace=True)
trim_df['gwarea'].fillna(method = 'bfill', inplace=True)
trim_df['mountains_mean'].fillna(value = trim_df['mountains_mean'].min(), inplace=True)
trim_df['bdist2'].fillna(method = 'bfill', inplace=True)

Now check if any missings are left in any feature:

No NaNs? Good lest go on to creating the test, validation and train set.

# 2A.5

Now we will create our `test_set`, `val_set` and `train_set`. Our `test_set` will be all observaitons from 2014 (`['year'] == 2014`). Our `val_set` will be all observations from 2013(`['year'] == 2013`). Oyr training set will be everything else (1989-2012).

In [ ]:
test_idx = trim_df[trim_df[...] == ...].index # get the indecies from all observation in year 2014
val_idx = trim_df[trim_df[...] == ...].index # get the indecies from all observation in year 2013
not_train_idx = np.append(test_idx,val_idx)

test_set = trim_df.loc[test_idx] # use the 2014 indecies to make a test set
val_set = trim_df.loc[val_idx] # use the 2013 indecies to make a validation set
train_set = trim_df.drop(index = not_train_idx) # Creat  training set containing non of the indecies from 2013  or 2014.


You can check your three new sets with whatevet way you think might convince you that everything is in order.

# 2A.6
Now we almost done. Lest generate some basline models. Specifically the Persistence model, the Ratio model and the Random model. Agian, model is here used in the most generous way possible. 

The Persistence "model" simply implies that we use our last observation as prediction for all future years. In essence the sentiment behind the Persistence model is: If some gid cell experienced conflict this year, I bet it will do so next year, and the year after and so on. Given one or two furture years (and the nature of conflict) this is actually not a bad bet, but it is a very simple one.

The Ratio "model" is slightly more sophisticated (not a lot though). As the name implies we here use the ratio of conflicts across a given cells years as predictor for all future conflicts. The ratio is per definition between 0 and 1 so we can treat it as a probability. Thus we simply set a threshold - conventionally 0.5 but it could be whatever we find prudent - and convert our ratios/probabilities to binary classes.

Neither the Persistence nor the Ratio model are particularly good - they are hardly models. But when/if we create some highly sophisticated monstrosity of a machine learning ensamble, we want to be sure its results are actually better then what we could get with one of these very simple baseline models - at least before we start showing it off.

Lastly, the Random "model" simply randomly predicts 0 or 1 for any cell. Really no usefull information is used here. It is just as a reference; how good or bad is some model compared to random guesses? (note: the random base line can often be analytically derived, but we will make it explicit here for the fun of it)

To measure how well we do here, we will use the simple evaluation metric `accuracy`. Denoting "true positives" TP, "false positives" FP, "true negatives" TN and "false negatives" FN accuracy is given by:

$$
\frac{TP + TN}{TP + TN + FP + FN} = \frac{P_c}{N}
$$

As shown we can also see it as all correct predictions $P_c$ divided by the number of observations $N$. Thus accuracy simply denotes the ratio of correctly classified observation out of all observaitons. Since we do not really train anything here, we will just eveluate directly on the training set. (However, we will still need to create the ratio "feature" in the val- and test_set for future references). 

So first, create a list of all your targets: `['binary_best_t1', 'binary_best_t2', 'binary_best_t3', 'binary_best_t4', 'binary_best_t5']`

In [ ]:
ys = ...

Use the following code to calculate the accuracy for the Persistence model across all targets:

`((train_set['binary_best'] == train_set[y])).sum()/ train_set.shape[0]`

In [ ]:
pm_acc_list = []
print('Persistence model: train set\n')
for y in ys:
    
    acc = ...
    print(f'{y} acc: {acc:.4f}')
    pm_acc_list.append(acc)


**Does the acc increase or decrease as we move forward thorugh the years?**   
**Can you see how the acc measure is related to the math above?**

Now lets create the ratio_best feature. As mentioned we are only gonna use the one in the train_set now, but lest create it in all our sets while we are at it:

In [ ]:
train_set['ratio_best'] = train_set.groupby('gid')['binary_best'].transform(np.mean)
test_set['ratio_best'] = test_set.groupby('gid')['binary_best'].transform(np.mean)
val_set['ratio_best'] = val_set.groupby('gid')['binary_best'].transform(np.mean)

Lets plot a year or two of ratio_best as a sanity check. The plots should be the same across years and reflect the general conflict level of the different cells.

Now, in the code below set `threshold = 0.5` and insert the acc calculation (from above), now with the object `threshold_ratio` instead of `train_set['ratio_best']`.

In [ ]:
rm_acc_list = []
print('ratio model: train set\n')
for y in ys:
    
    threshold = 0.5
    
    threshold_ratio = (train_set['ratio_best']>threshold)*1
    
    acc = ...
    print(f'{y} acc: {acc:.4f}')
    rm_acc_list.append(acc)


In the loops above we created a list for respectively the Persistence model and the Ratio model. The code below will plot the acc trends for the Persistence and Ratio model:

In [ ]:
t = np.arange(1,len(rm_acc_list)+1,1)
plt.plot(t,pm_acc_list, '-o', label = 'persitance')
plt.plot(t,rm_acc_list, '-o', label = 'ratio')

plt.xticks(t)
plt.ylabel('Acc')
plt.xlabel('years into the furture')

plt.legend()
plt.show()

**Which one is doing best?**   
**Are you surprised they get so high acc scores?**  
**Can you explain/ponder about what might be the reason for these very high scores?**  

Lest for comaprison look at the Random "model":

In [ ]:
rdm_acc_list = []
print('ratio model: train set\n')
for y in ys:
        
    random_binary = np.random.randint(0,2,train_set.shape[0])
    
    acc = ((random_binary == train_set[y])).sum()/ train_set.shape[0]
    print(f'{y} acc: {acc:.4f}')
    rdm_acc_list.append(acc)

**What does the acc score of the random model tell you?**

So we see that the Persistence model and the Ratio model (which are both barely models) are not without merit. At least they do way better than the random model. Also, not only are they important baselines when we evaluate our more complex models - but just as importantly, they can be act as strong features themselves in these more complex models. If this is not clear by now, it will be later on. 

# 2A.7
pickle your three new dataframes `test_set`, `train_set` and `val_set` as `test_set.pkl`, `train_set.pkl` and `val_set.pkl` (feel free to pickel trim_df as well if you want to).

In [ ]:
#pickle test

file_name = ...
output = open(file_name, 'wb')
pickle.dump(..., output)
output.close()

#pickle train

file_name = ...
output = open(file_name, 'wb')
pickle.dump(..., output)
output.close()

#pickle val

file_name = ...
output = open(file_name, 'wb')
pickle.dump(..., output)
output.close()

Load our pickles and test that everything is correct - whatever it needs to convince you.

Done.